In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
#export
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
from google.cloud import bigquery

In [ ]:
# default_exp schema

# schema

> Functionality related to dealing with schemas in BigQuery.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export


def get_schema(table_id: str) -> list:
    """
    Get schema given a table_id.
    """
    client = bigquery.Client()
    table = client.get_table(table_id)
    schema = table.schema
    return schema



In [ ]:
# for example get a schema for a sample table
schema = get_schema('bigquery-public-data.samples.shakespeare')
schema

[SchemaField('word', 'STRING', 'REQUIRED', 'A single unique word (where whitespace is the delimiter) extracted from a corpus.', ()),
 SchemaField('word_count', 'INTEGER', 'REQUIRED', 'The number of times this word appears in this corpus.', ()),
 SchemaField('corpus', 'STRING', 'REQUIRED', 'The work from which this word was extracted.', ()),
 SchemaField('corpus_date', 'INTEGER', 'REQUIRED', 'The year in which this corpus was published.', ())]

In [ ]:
# tests
assert len(schema) == 4
assert [col.name for col in schema] == ['word', 'word_count', 'corpus', 'corpus_date']

In [ ]:
#export


def dtype_to_bqtype(dtype, default_type: str = 'STRING') -> str:
    """
    Convert from a pandas type to BigQuery type.
    """
    bqtype = default_type
    if dtype == 'int64':
        bqtype = 'INTEGER'
    return bqtype



In [ ]:
#export


def df_to_bq_schema(df: pd.DataFrame) -> list:
    """
    Read a pandas DF and return a BigQuery schema for that DF.
    """
    schema = []
    for col, dtype in df.dtypes.iteritems():
        schema.append(bigquery.SchemaField(col, dtype_to_bqtype(dtype)))
    return schema



In [ ]:
# example usage

# make a df
df = pd.DataFrame([[1, 2], [1, 2], [1, 2]], columns=['col1', 'col2'])
print(df.shape)
print(df)

# get bq schema
schema = df_to_bq_schema(df)
print(schema)

(3, 2)
   col1  col2
0     1     2
1     1     2
2     1     2
[SchemaField('col1', 'INTEGER', 'NULLABLE', None, ()), SchemaField('col2', 'INTEGER', 'NULLABLE', None, ())]


In [ ]:
#hide

# tests 
assert len(schema) == 2
assert [col.name for col in schema] == ['col1', 'col2']

In [ ]:
#export


def schema_diff(old_schema, new_schema) -> list:
    """
    Compare two BigQuery schemas and return a list of differences.
    """
    old_schema_dict = {col.name:col for col in old_schema}
    new_schema_dict = {col.name:col for col in new_schema}
    diffs = []
    for col in new_schema_dict:
        if col not in old_schema_dict:
            diffs.append(('add', new_schema_dict[col]))
        elif new_schema_dict[col] != old_schema_dict[col]:
            diffs.append(('update', old_schema_dict[col], new_schema_dict[col]))
    for col in old_schema_dict:
        if col not in new_schema_dict:
            diffs.append(('drop', old_schema_dict[col]))
    return diffs



In [ ]:
#hide

# tests
old_schema = [bigquery.SchemaField("col1", "STRING"),bigquery.SchemaField("col2", "STRING")]
new_schema = [bigquery.SchemaField("col2", "STRING"),bigquery.SchemaField("col3", "STRING")]
expected_diff = [('add', bigquery.SchemaField("col3", "STRING")),('drop', bigquery.SchemaField("col1", "STRING"))]
actual_diff = schema_diff(old_schema, new_schema)
assert actual_diff == expected_diff

In [ ]:
#export


def update_bq_schema(bq_client, table_id: str, diffs: list, print_info: bool = True) -> bool:
    """
    Given a list of diffs and a table_id add any new columns to table.
    """
    # only do work if some 'add's in diffs
    if 'add' in [diff[0] for diff in diffs]:
        table = bq_client.get_table(table_id)
        current_schema = table.schema
        current_schema_col_names = [col.name for col in current_schema]
        for diff in diffs:
            if diff[0] == 'add':
                bq_schema_field = diff[1]
                col_name = bq_schema_field.name
                if col_name not in current_schema_col_names:
                    if print_info:
                        print(f'adding {bq_schema_field} to {table_id}')
                    new_schema = current_schema[:]
                    new_schema.append(bq_schema_field)
                    table.schema = new_schema
                    table = bq_client.update_table(table, ["schema"])
    return True



In [ ]:
#export


def update_df_schema(bq_client, table_id: str, diffs: list, df: pd.DataFrame, print_info: bool = True) -> pd.DataFrame:
    """
    Given a list of diffs add any columns expected but not found in df.
    """
    # only do work if some 'drops's in diffs
    if 'drop' in [diff[0] for diff in diffs]:
        for diff in diffs:
            if diff[0] == 'drop':
                col_name = diff[1].name
                if col_name not in df.columns:
                    if print_info:
                        print(f'adding {col_name} to df')
                    df[col_name] = None
    table = bq_client.get_table(table_id)
    bq_schema = table.schema
    bq_schema_col_names = [col.name for col in bq_schema]
    df = df[bq_schema_col_names]
    return df

